In [230]:
import numpy as np
import scipy.optimize 
import copy

class MyRobot():
    
    def __init__(self,kinematic_chain, mass, joint_limits):
        self.tool=[]
        self.world_frame=np.eye(4)
        self.home_position=np.array([0,0,0,0,0,0])
        self.joints=kinematic_chain[:,2]
        self.pastjoints=self.joints
        self.joint_limits=np.deg2rad(joint_limits)
        self.kinematic_chain=kinematic_chain
        self.mass=mass[0,:]
        self.com=mass[1:7,0:3]
        self.wrench=[]
        self.dof=len(kinematic_chain)
    
    def __len__(self):
        return self.dof
    
   
    def transform(self,alpha,a,theta,d):
        
        sinalpha=np.sin(alpha)
        cosalpha=np.cos(alpha)
        sintheta=np.sin(theta)
        costheta=np.cos(theta)
        t=np.array(
            [
                [costheta, -sintheta, 0, a],
                [sintheta*cosalpha, costheta*cosalpha, -sinalpha, -sinalpha*d],
                [sintheta*sinalpha, costheta*sinalpha, cosalpha, cosalpha*d],
                [0, 0, 0, 1],
            
            ]
          )
        
        return t
    
    def invtransform(self, alpha,a,theta,d):
        
        sinalpha=np.sin(alpha)
        cosalpha=np.cos(alpha)
        sintheta=np.sin(theta)
        costheta=np.cos(theta)
        r=np.array(
            [
                [costheta, -sintheta, 0],
                [sintheta*cosalpha, costheta*cosalpha, -sinalpha],
                [sintheta*sinalpha, costheta*sinalpha, cosalpha],
            ]
        )
        
        t=np.array(
            [
                [a],
                [-sinalpha*d],
                [cosalpha*d],
            ]
        )
        
        invt=np.concatenate((r.T,-np.dot(r.T,t)),axis=1)
        invt=np.concatenate((invt,[[0,0,0,1]]),axis=0)
        
        return invt
    
    
    def random_joints(self,no_output=False):
        #if output is true output random value 
        #by default the method output the random joints value
        #if don't want any out put, give the argument no_output=1
        q=np.random.uniform(low=self.joint_limits[0], high=self.joint_limits[1])
        if no_output:
            self.joints=q
            return None
        else :
            return q

    def g_i(self,i):
        g_b=np.array([0,0,-9.81,0])
        t=np.eye(4)
        for j in range(i):
            alpha=self.kinematic_chain[i,0]
            a=self.kinematic_chain[i,1]
            theta=self.joints[i]
            d=self.kinematic_chain[i,3]
            t=np.dot(self.invtransform(alpha,a,theta,d),t)
        g_i=np.dot(t,g_b)
        return g_i
    
    def get_torque(self,wrench):
        Moment=np.zeros((6,3))
        Force=np.zeros((6,3))
        force=np.array(wrench[:3])
        moment=np.array(wrench[-3:])
        m=self.mass
        com=self.com
        g_b=np.array([0,0,-9.81,0]); #gravity in world coordinate 
        M=np.zeros((7,7,3))
        F=np.zeros((7,7,3))
        M_g=np.zeros((7,7,3))
        for i in range(5,-1,-1):
            
            g_i=self.g_i(i)[:3]
            alpha=self.kinematic_chain[i,0]
            a=self.kinematic_chain[i,1]
            theta=self.joints[i]
            d=self.kinematic_chain[i,3]
            transform=self.transform(alpha,a,theta,d)
            rotation=transform[:3,:3]
            if i==5:
                F[i+1,i+1]=force
                M[i,i+1]=moment
                F[i,i]=m[i]*g_i+F[i+1,i+1]
            else:
                F[i,i]=m[i]*g_i+np.dot(rotation,F[i+1,i+1]).T
                M[i,i+1]=np.dot(rotation,M[i+1,i+1])+np.cross(com[i],np.dot(rotation,F[i+1,i+1]))
            M_g[i,i]=m[i]*np.cross(self.com[i],g_i)
            M[i,i]=M_g[i,i]+M[i,i+1]
            
            Moment[i]=M[i,i]
            Force[i]=F[i,i]
        
        return Moment[:,2] #only return the torque in the direction of the z axis
   
    def matrix(self,q):
        
        t_world=np.eye(4)
        t=[]
        for i in range(len(self)):
            alpha=self.kinematic_chain[i,0]
            a=self.kinematic_chain[i,1]
            theta=q[i]
            d=self.kinematic_chain[i,3]
            t.append(self.transform(alpha,a,theta,d))
            
        t_tool=np.eye(4) 
            
        return t    

   
    def dq(self,q):
        #only include joint compliance caused by joint stiffness and torque
        torque=self.get_torque(self.wrench)
        #define joint stiffness value 
        stiffness=np.array([0.0005,0.0005,0.0005,0.0005,0.0005,0.0005]) 
        #calculate joint compliance based on linear model:
        # delta_q=stiffness*torque
        delta_q= torque*stiffness
        return delta_q
    
    

    def fk(self,q):
        
        q=q+self.dq(q)
        self.pastjoints=self.joints
        self.joints=q
        #q=np.deg2rad(q)
        t=np.eye(4)
        for i in range(len(self)):
            t=np.dot(t,self.matrix(q)[i])
        return t
        
    
    def ik(self,pose):
        x0 = self.joints
        result = scipy.optimize.least_squares(
            fun=ik_cost_function, x0=x0, bounds=self.joint_limits, args=(pose,self)
        )  # type: scipy.optimize.OptimizeResult

        if result.success:  
            actual_pose = self.fk(result.x)
            if np.allclose(actual_pose, pose, atol=1e-1):
                return result.x
        return None
    
     
    
    @classmethod
    def importrobot(cls, dhparameters,massparameters,jointlimits):
        """Construct Robot from parameters."""
        return cls(kinematic_chain=dhparameters,mass=massparameters,joint_limits=jointlimits)
    
def ik_cost_function(q,pose,robot):
        actual_pose=robot.fk(q)
        diff=np.abs(actual_pose-pose)
        #fun must return at most 1-d array_like.
        return diff.ravel()

    


Define Robot kinematic chain, joint limits, mass& center of mass

In [231]:
def kuka_90_R3100()-> np.ndarray:
    return np.array(
        [ # alpha, a, theta, d
            [0, 0, 0, 0.675],
            [np.pi/2, 0.35, 0, 0],
            [0, 1.35, 0, 0],
            [-np.pi/2, -0.041, 0, 1.4],
            [np.pi/2, 0, 0, 0],
            [-np.pi/2, 0, 0, 0.215],
        ]
    )

def kuka_90_R3100_mass():
    
    return np.array(
        [
            [15,10,10,6,3,1], #mass of link 1 to 6
            [0.3,0.1,0.1,0,0,0], #center of mass of link1
            [0.2,0.1,0.1,0,0,0], #center of mass of link2
            [0.3,0.3,0.1,0,0,0], #center of mass of link3
            [0.1,0.2,0.1,0,0,0], #center of mass of link4
            [0.1,0.3,0.1,0,0,0], #center of mass of link5
            [0.1,0.2,0.1,0,0,0], #center of mass of link6
        ]
    )

def kuka_90_R3100_joint_limits():
    return np.array(
        [
            #lower bound must smaller than upper bound 
            [0,0,0,0,0,0], #lower bound for joints 1 to 6
            [180,70,80,100,160,360], #upper bound for joints 1 to 6
            
        ]
    )

Import robot and create MyRobot object 

In [232]:
kuka=MyRobot.importrobot(kuka_90_R3100(),kuka_90_R3100_mass(),kuka_90_R3100_joint_limits())

Define force and torque at robot TCP

In [274]:
#before use fk() and ik() wrench must be defined first
#define the external force and torque at TCP
forces = [0, 0, 10]
torques = [5, 0, 0]
wrench = [*forces, *torques]
kuka.wrench=wrench
kuka.joints=np.deg2rad([10,10,20,30,20,120])
print(kuka.get_torque(wrench))


[ 35.08365486  32.67612479 -41.02303131  -7.39340888   2.06649408
   0.31638808]


Joint compliance due to external wrench and robot link mass 

In [234]:
dq=kuka.dq(kuka.joints)
print(np.rad2deg(dq))

[ 1.00507268  0.93610202 -1.17522328 -0.21180556  0.05920069  0.00906385]


Forward kinematic 

In [237]:
joints=np.deg2rad([10,10,20,30,20,20])
pose=kuka.fk(joints)
print(pose)

[[ 0.21425196 -0.70211345 -0.6790676   0.7831344 ]
 [ 0.78936411  0.53393874 -0.30300778  0.11306174]
 [ 0.57532634 -0.47111158  0.66862058  2.26827035]
 [ 0.          0.          0.          1.        ]]


Inverse Kinematic 

In [247]:
joints=np.deg2rad([10,10,20,30,20,20])
ik_pose=kuka.fk(joints)
ik_joints=np.rad2deg(kuka.ik(ik_pose))
print(ik_joints)

[10.84305056 10.85059837 18.88238653 29.72471339 20.0149379  20.01370884]



Model verification (compare fk and ik with pybotics)


In [248]:
import pybotics 
from pybotics.robot import Robot
from pybotics.kinematic_chain import KinematicChain, MDHKinematicChain
from pybotics.tool import Tool

define robot objects

In [254]:
pyrobot=Robot.from_parameters(kuka_90_R3100())
myrobot=MyRobot.importrobot(kuka_90_R3100(),kuka_90_R3100_mass(),kuka_90_R3100_joint_limits())
forces = [0, 0, 10]
torques = [5, 0, 0]
wrench = [*forces, *torques]
myrobot.wrench=wrench

set random joint values and perform forward kinematic 

In [255]:
low_limit=kuka_90_R3100_joint_limits()[0]
high_limit=kuka_90_R3100_joint_limits()[1]
position_my=[]
position_py=[]
joints=[]
iksolution_my=[]
iksolution_py=[]
for i in range(10):
    q=np.random.uniform(low=low_limit, high=high_limit)
    q=np.deg2rad(q)
    pose_pyrobot=pyrobot.fk(q)
    pose_myrobot=myrobot.fk(q)
    position_my.append(pose_myrobot[:-1,-1])
    position_py.append(pose_pyrobot[:-1,-1])


compare the position 

In [256]:
display(position_my)
display(position_py)

[array([ 0.00496796, -0.11837138,  1.73893206]),
 array([3.10807785e-04, 1.05280064e-01, 1.73754353e+00]),
 array([ 0.02455711, -0.0649083 ,  1.69763712]),
 array([ 0.19728255, -0.09155968,  1.04991206]),
 array([0.2654677 , 0.18581769, 1.92866953]),
 array([0.06654076, 0.11420819, 0.59469015]),
 array([0.06430091, 0.17795404, 1.00792223]),
 array([0.08320479, 0.15643937, 1.21421097]),
 array([-0.03864577,  0.04708823,  1.38242578]),
 array([ 0.26048755, -0.07368473,  1.3856979 ])]

[array([ 0.00498778, -0.11154744,  1.71847703]),
 array([-6.03077699e-04,  1.07435445e-01,  1.70785103e+00]),
 array([ 0.02658401, -0.0565085 ,  1.68170163]),
 array([ 0.20679865, -0.08991946,  1.03586367]),
 array([0.27060271, 0.18328669, 1.92314591]),
 array([0.06557797, 0.1337877 , 0.58365468]),
 array([0.06323873, 0.19066282, 0.98435836]),
 array([0.07487726, 0.15635038, 1.20218816]),
 array([-0.04336143,  0.04875431,  1.37258994]),
 array([ 0.24833568, -0.06701976,  1.37190624])]